In [1]:
import numpy as np
import pandas as pd
import string
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
import nltk
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
#Run the below piece of code for the first time
#nltk.download('stopwords')

In [2]:
message_data = pd.read_csv("emailDataset.csv",encoding = "latin")
message_data.head()

,emailId,tag,mail-message
0,1,boss,"Dear Tom, I can't thank you enough for forward..."
1,2,possy,"Hi Linda, How's it going? Sorry I haven't been..."
2,3,possy,"Hey, Susi, whats wrong with you? Why aren't yo..."
3,4,boss,"Dear Barylior, I will need your opinion on how..."
4,5,boss,"Dear Liz,I really appreciate your understandin..."


In [3]:
# message_data = message_data.drop(['Unnamed: 3', 'Unnamed: 4', 'Unnamed: 5'], axis = 1)


In [4]:
message_data

,emailId,tag,mail-message
0,1,boss,"Dear Tom, I can't thank you enough for forward..."
1,2,possy,"Hi Linda, How's it going? Sorry I haven't been..."
2,3,possy,"Hey, Susi, whats wrong with you? Why aren't yo..."
3,4,boss,"Dear Barylior, I will need your opinion on how..."
4,5,boss,"Dear Liz,I really appreciate your understandin..."
5,6,boss,"Dear Chris,I just wanted to drop you a note to..."
6,7,boss,"Dear Ryanna, Thank you so much for the generou..."
7,8,possy,Brian ! like to talk to people from other part...
8,9,boss,"Dear Ms. Smith, This is to bring to your notic..."
9,10,possy,"Hey Lucie, since we last talked, I started dat..."


In [5]:
 message_data.groupby('tag').describe()


emailId                                              
        count  mean       std  min   25%   50%    75%   max
tag                                                        
boss     10.0   9.0  5.374838  1.0  5.25   8.0  13.25  17.0
possy    10.0  12.0  6.324555  2.0  8.50  12.5  17.25  20.0

In [6]:
message_data_copy = message_data['mail-message'].copy()

message_data_copy

0     Dear Tom, I can't thank you enough for forward...
1     Hi Linda, How's it going? Sorry I haven't been...
2     Hey, Susi, whats wrong with you? Why aren't yo...
3     Dear Barylior, I will need your opinion on how...
4     Dear Liz,I really appreciate your understandin...
5     Dear Chris,I just wanted to drop you a note to...
6     Dear Ryanna, Thank you so much for the generou...
7     Brian ! like to talk to people from other part...
8     Dear Ms. Smith, This is to bring to your notic...
9     Hey Lucie, since we last talked, I started dat...
10    Hello Sir, I am a student of yours from UCLA. ...
11    Emy . we are not friends anymore. On my Birthd...
12    Hi Weasel! Hope this mail finds you in the bes...
13    Dear Sir,  Please accept this mail as a notice...
14    Hi my friend
 I havent seen you for a lot tim...
15    Dear Ms. Lee, I am writing to request consider...
16    Dear Mr./Ms. {Recipients Name}, I am writing ...
17    Liza, Being your sister is the greatest bl

In [7]:
def text_preprocess(text):
    text = text.translate(str.maketrans('', '', string.punctuation))
    text = [word for word in text.split() if word.lower() not in stopwords.words('english')]
    return " ".join(text)

In [8]:
def stemmer (text):
    text = text.split()
    words = ""
    for i in text:
            stemmer = SnowballStemmer("english")
            words += (stemmer.stem(i))+" "
    return words

In [9]:
message_data_copy = message_data_copy.apply(text_preprocess)
message_data_copy = message_data_copy.apply(stemmer)    # New Line
# message_data_copy

In [10]:
vectorizer = TfidfVectorizer("english")
message_mat = vectorizer.fit_transform(message_data_copy)


c:\users\kids\appdata\local\programs\python\python38\lib\site-packages\sklearn\utils\validation.py:68: FutureWarning: Pass input=english as keyword args. From version 0.25 passing these as positional arguments will result in an error
  warnings.warn("Pass {} as keyword args. From version 0.25 "


In [11]:
# Normalizing length.
# The code below works ,but it doesn't have any efect on the accuracy . Use it carefully
# message_data['length'] = message_data['mail-message'].apply(len)
# message_data.head()
# length = message_data['length'].to_numpy()

# new_mat = np.hstack((message_mat.todense(),length[:, None]))

In [12]:
message_train, message_test, spam_nospam_train, spam_nospam_test = train_test_split(message_mat, 
                                                        message_data['tag'], test_size=0.3, random_state=20)

In [13]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

Spam_model = LogisticRegression(solver='liblinear', penalty='l1')
Spam_model.fit(message_train, spam_nospam_train)
pred = Spam_model.predict(message_test)
accuracy_score(spam_nospam_test,pred)

print("The prediction is :", pred)
print("Accuracy Score: ", format(accuracy_score(spam_nospam_test,pred)*100)+" %")

The prediction is : ['boss' 'boss' 'boss' 'boss' 'boss' 'boss']
Accuracy Score:  66.66666666666666 %


In [14]:
# ## My Try 
# ## 1. Take sample email
# # 2. Preprocess it
# # 3. Vectorize it
# # 4. Put it in the model
# test_email=message_data_copy[0] 
# test_email
# test_email=[test_email]


In [15]:
dummyEmail = ["Hey, Susi, whats wrong with you? Why aren't you "]
dummyEmailMatrix= vectorizer.transform(dummyEmail) 
dummyEmailMatrix
predict= Spam_model.predict(dummyEmailMatrix)
print("The prediction is :", predict)

The prediction is : ['boss']
